In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
batch_size = 4

train_transforms = transforms.Compose(

    [
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((512,512)),
        transforms.ToTensor(),
    ]
)

In [4]:
path1 = '/root/data0/CXR_IMG/CXR_PT'
path2 = '/root/data0/CXR_LABEL/CXR_PT'
cxr1 = os

In [5]:
path_img = '/root/data0/CXR_IMG/'

In [6]:
trainset = torchvision.datasets.ImageFolder(root = path_img, transform = train_transforms)

In [7]:
dataset_size = len(trainset)
dataset_idxs = list(range(dataset_size))

np.random.shuffle(dataset_idxs)

val_split_idx = int(np.floor(0.2*dataset_size))
train_idx, valid_idx = dataset_idxs[val_split_idx:],dataset_idxs[:val_split_idx]


train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_iter = torch.utils.data.DataLoader(trainset,
                                          shuffle = False,
                                          batch_size = batch_size,
                                          sampler = train_sampler,
                                          num_workers = 1)

valid_iter = torch.utils.data.DataLoader(trainset,
                                          shuffle = False,
                                          batch_size = batch_size,
                                          sampler = valid_sampler,
                                          num_workers = 1)

In [8]:
path_img = '/root/data0/CXR_IMG/'
orig_set = torchvision.datasets.ImageFolder(root = path_img, transform = train_transforms)
n = len(orig_set)

n_valid = int(0.2 * n)
validset = torch.utils.data.Subset(orig_set, range(n_valid))
trainset = torch.utils.data.Subset(orig_set,range(n_valid,n))
train_iter = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=False)
valid_iter= torch.utils.data.DataLoader(validset, batch_size = batch_size, shuffle=False)


In [9]:
path_seg = '/root/data0/CXR_LABEL/'
trainset_seg = torchvision.datasets.ImageFolder(root = path_seg, transform = train_transforms)
train_iter_seg = torch.utils.data.DataLoader(trainset_seg, batch_size = batch_size, shuffle=True)


In [10]:
dataiter=iter(train_iter)
img,lbl = dataiter.next()


In [11]:
print(lbl.shape)

torch.Size([4])


In [12]:
import torchvision.models as models

In [13]:
net = models.efficientnet_b0()
net

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [14]:
net.features[0][0] = nn.Conv2d(1,32,kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

In [15]:
net.classifier[1] = torch.nn.Linear(1280,2)

In [16]:

net = net.to(device)
net

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [17]:
lr = 1e-4
loss = torch.nn.CrossEntropyLoss()
alg = torch.optim.AdamW(net.parameters(),lr=lr)
net.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [18]:
loss_train = np.array([])
accs_train = np.array([])
accs_valid = np.array([])
num_epoch = 50
for epoch in range(num_epoch):
    i = 0
    l_epoch = 0
    correct = 0
    net.train()
    for X,y in train_iter:
        i = i+1
        X,y = X.to(device), y.to(device)
        y_hat = net(X) # forward
        correct += (y_hat.argmax(dim=1)==y).sum()
        l = loss(y_hat,y)
        l_epoch += l
        alg.zero_grad()
        # backpropagation
        l.backward()
        alg.step()
    loss_train = np.append(loss_train,l_epoch.cpu().detach().numpy()/i)
    accs_train = np.append(accs_train,correct.cpu()/len(trainset))
    
    corrct = 0
    net.eval()
    for X,y in valid_iter :
        X,y = X.to(device),y.to(device)
        y_hat = net(X)
        correct += (y_hat.argmax(dim=1)==y).sum()
    accs_valid = np.append(accs_valid,correct.cpu()/len(validset))
        
        
    if epoch % 10 == 0 :
        print('epoch: %d' %(epoch))
        print('train loss: ', loss_train[-1])
        print('train accuracy: ', accs_train[-1])
        print('valid accuracy: ', accs_valid[-1])

RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 31.75 GiB total capacity; 1.86 GiB already allocated; 24.75 MiB free; 1.94 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# valid set 결과 확인

i=0
y_list= np.array([])
y_hat_list = np.array([])

for X,y in valid_iter:
    X = X.to(device)
    y_hat = net(X)
    y_hat= y_hat.argmax(dim=1)
    
    y_list = np.append(y_list,y)
    y_hat_list = np.append(y_hat_list,y_hat.cpu().numpy())

    if i==0:
        for n in range(4):
            plt.figure(dpi=128)
            plt.imshow(X[n].cpu().squeeze(),cmap='gray')
            plt.title('label : %s \n predcition: %s' %(tgtnames[y[n]],tgtnames[y_hat[n]]))
            plt.pause(.0001)
    i = i+ 1
    
    

In [ ]:
# confusion matrix

from sklearn.metrics import(
    classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)

cm = confusion_matrix(
    y_list,
    y_hat_list,
    normalize = 'true',
)
disp = ConfusionMatrixDisplay(
    confusion_matrix = cm,
    display_labels = tgtnames,
)

disp.plot(ax = plt.subplots(1,1,facecolor = 'white')[1], cmap = 'gray')

In [ ]:
# roc curve

from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score


fpr_tr , tpr_tr , ths_tr = roc_curve(y,y_hat)
auc_tr = auc(fpr_tr , tpr_tr)

fig = plt.figure(figsize = (8,4))
plt.subplot(121)
plt.plot( fpr_tr , tpr_tr, lw = 3 , label = 'ROC curve (area = %0.2f)' % auc_tr)
plt.plot([0,1] , [0,1] , color = 'skyblue', lw = 2 , linestyle = '--')
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.legend(loc = 'lower right' , prop = {'size' : 10})
plt.show()

segmentation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage import io
from skimage.color import rgb2gray
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import Subset
from torch.utils.data import DataLoader


from skimage import io
from skimage import color
from skimage.feature import (
    match_descriptors,
    corner_harris,
    corner_peaks,
    ORB,
    plot_matches)

from skimage.transform import(
    rescale,
    resize,
    downscale_local_mean,
    rotate
)

import glob


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
batch_size = 4

In [ ]:

import glob

CXR_label_NM = glob.glob('/root/data0/CXR_LABEL/CXR_NM/LABEL_NM.png')# 정상 레이블

CXR_label_PT = glob.glob('/root/data0/CXR_LABEL/CXR_PT/*.png') # 병변 레이블


CXR_list_NM = glob.glob('/root/data0/CXR_IMG/CXR_NM/*.png')# 정상

CXR_list_PT = glob.glob('/root/data0/CXR_IMG/CXR_PT/*.png')# 병변

In [ ]:
nm = [
    {
        'img' : os.path.join(CXR_list_NM[i]),
        'seg' : os.path.join(CXR_label_NM[0])
    }for i in range(len(CXR_list_NM))
]

pt = [
    {
        'img' : os.path.join(CXR_list_PT[i]),
        'seg' : os.path.join(CXR_label_PT[i])
    }for i in range(len(CXR_list_PT))
]

In [ ]:
pair_dataset = nm + pt

In [ ]:
#노말1400 pt 744 4:1나눠주기

Ntr_nm = 1120
Nvd_nm = 280

train_nm = nm[:Ntr_nm]
valid_nm = nm[-Nvd_nm:]


Ntr_pt = 595
Nvd_pt = 149

train_pt = pt[:Ntr_pt]
valid_pt = pt[-Nvd_pt:]


In [ ]:
pair_tr = train_nm + train_pt
pair_vd = valid_nm + valid_pt
print(pair_tr)

In [ ]:
import torchvision.models as models
model = models.segmentation.fcn_resnet50(weights_backbone=None)


In [ ]:
model.backbone.conv1 = torch.nn.Conv2d(1,64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [ ]:
model.classifier[4] = torch.nn.Conv2d(512,2, kernel_size=(1, 1), stride=(1, 1))

In [ ]:
model.to(device)

In [ ]:
trainloader = DataLoader(pair_tr, batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)
validloader = DataLoader(pair_vd, batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, weight = None, size_average = True):
        super(DiceLoss, self).__init__()
        
    def forward(self, inputs, targets, smooth = 1):
        inputs = F.sigmoid(inputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()
        dice = (2.* intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        
        return 1-dice

In [ ]:
lr = 1e-4
num_epochs = 50
l = DiceLoss()
loss = torch.nn.CrossEntropyLoss()
alg = torch.optim.SGD(model.parameters(),lr=lr)

In [ ]:
loss_train = np.array([])
accs_train = np.array([])
accs_valid = np.array([])
num_epoch = 30
batch_size=4

for epoch in range(num_epochs):
    i = 0
    l_epoch = 0
    correct = 0
    model.train()
    for batch in trainloader:
        data = batch['img']
        target= batch['seg']
        i = i + 1
        
        X = torch.zeros((batch_size,1,512,512))
        Y = torch.zeros((batch_size,1,512,512))

        for j in range(batch_size):
            X1 = Image.open(data[j])
            X1 = torch.tensor(resize(np.array(X1),(512,512)))
            X[j,0] = X1
            X = X.to(device)

            Y1 = Image.open(target[j])
            Y1 = torch.tensor(resize(np.array(Y1),(512,512)))
            Y[j,0] = Y1.to(device)
            Y = Y.to(device)
        y_hat = model(X)
        y_hat = y_hat['out']
#         y_hat = y_hat.to(device)
        y = np.squeeze(Y,axis=1)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        correct += (y_hat.argmax(dim=1)==Y).sum()
        l = loss(y_hat,y)
        l_epoch += l
        alg.zero_grad()

        l.backward()
        alg.step()
    loss_train = np.append(loss_train,l_epoch.cpu().detach().numpy()/i)
    accs_train = np.append(accs_train,correct.cpu()/len(pair_tr))
    
    corrct = 0
    model.eval()
    for batch in validloader:
        data = batch['img']
        target= batch['seg']

    
        X = torch.zeros((batch_size,1,512,512))
        Y = torch.zeros((batch_size,1,512,512))

        for j in range(batch_size):
            X1 = Image.open(data[j])
            X1 = torch.tensor(resize(np.array(X1),(512,512)))
            X[j,0] = X1
            X = X.to(device)

            Y1 = Image.open(target[j])
            Y1 = torch.tensor(resize(np.array(Y1),(512,512)))
            Y[j,0] = Y1.to(device)
            Y = Y.to(device)
        y_hat = model(X)
        y_hat = y_hat['out']
        correct += (y_hat.argmax(dim=1)==Y).sum()
#         X,y = X.to(device),y.to(device)
#         y_hat = model(X)
#         y_hat = y_hat['out']
#         correct += (y_hat.argmax(dim=1)==y).sum()
    accs_valid = np.append(accs_valid,correct.cpu()/len(pair_vd))
        
        
    if epoch % 3 == 0 :
        
        print('epoch: %d' %(epoch))
        print('train loss: ', loss_train[-1])
        print('train accuracy: ', accs_train[-1])
        print('valid accuracy: ', accs_valid[-1])
   
    
